In [ ]:
%cd /content/drive/MyDrive/ri/code/data

In [ ]:
import re

In [ ]:
import itertools

In [ ]:
import nltk

In [ ]:
from nltk.corpus import stopwords

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
from nltk.stem import WordNetLemmatizer

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer

In [ ]:
import pandas as pd

In [ ]:
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
from collections import Counter

In [ ]:
#Data Understanding and Preparation

In [ ]:
## Import Libraries and Load Data

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ri/code/data/A1_standard.csv')

In [ ]:
print(df.head())

In [ ]:
## Data Information and Missing Values

In [ ]:
df.info()

In [ ]:
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
sns.heatmap(df.isnull(), cbar=False, cmap='viridis')

In [ ]:
plt.title('Missing Data Visualization')

In [ ]:
plt.show()

In [ ]:
# Handle Missing Values, given there are none but still

In [ ]:
if df['text'].isnull().sum() > 0:

In [ ]:
    df['text'] = df['text'].fillna('no_text')

In [ ]:
    # df.dropna(subset=['text'], inplace=True)

In [ ]:
## Remove Noise, Tokenize, and Normalize Text

In [ ]:
# Ensure nltk resources are downloaded (first time run)

In [ ]:
import nltk

In [ ]:
nltk.download('punkt')

In [ ]:
nltk.download('stopwords')

In [ ]:
nltk.download('wordnet')

In [ ]:
nltk.download('averaged_perceptron_tagger')

In [ ]:
nltk.download('vader_lexicon')

In [ ]:
# Improved dictionary for replacing abbreviations and unclear words

In [ ]:
replacement_dict = {

In [ ]:
    'eur': 'euro',

In [ ]:
    'mn': 'million',

In [ ]:
    'mln': 'million',

In [ ]:
    'oyj': 'company',

In [ ]:
    'pct': 'percent',

In [ ]:
    'hel': 'Helsinki',

In [ ]:
    'eurm': 'euro million'  # Assuming eurm means euro million

In [ ]:
}

In [ ]:
def preprocess_text(text):

In [ ]:
    # Remove HTML tags

In [ ]:
    text = re.sub(r'<.*?>', '', text)

In [ ]:
    # Replace hyphens with spaces (to maintain semantic integrity)

In [ ]:
    text = re.sub(r'-', ' ', text)

In [ ]:
    # Remove punctuation except for periods (which can help in maintaining sentence boundaries)

In [ ]:
    text = re.sub(r'[^\w\s\.]', '', text)

In [ ]:
    # Convert text to lowercase

In [ ]:
    text = text.lower()

In [ ]:
    # Tokenize text

In [ ]:
    tokens = word_tokenize(text)

In [ ]:
    # Lemmatization (Avoid lemmatizing negations to preserve their form)

In [ ]:
    lemmatizer = WordNetLemmatizer()

In [ ]:
    tokens = [lemmatizer.lemmatize(word) if not plural_related(word) else word for word in tokens]